# Задание 2: Фильтр Блума с счетчиком

In [1]:
%reset -f

In [2]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt
from collections import Counter
from get_hashes_module import get_hashes

In [3]:
class BloomFilterCount:
    def __init__(self, size, hash_count):
        self.size = size  # Размер битового массива
        self.hash_count = hash_count  # Количество хеш-функций
        self.bit_array = [0] * size  # Битовый массив, начальный вид - все биты равны 0
        self.element_count = 0

    def add(self, item):
        hashes = get_hashes(item, self.size, self.hash_count)
        for hash in hashes:
            self.bit_array[int(hash,2)] += 1  # Устанавливаем соответствующие биты в 1
        self.element_count += 1

    def check(self, item):
        hashes = get_hashes(item, self.size, self.hash_count)
        counter = Counter(hashes)

        for hash, count in counter.items():
            if self.bit_array[int(hash,2)] < count:
                return False  # Если хотя бы один бит не установлен, элемент точно отсутствует
        return True  # Если все биты установлены, элемент, возможно, присутствует
    
    # должны проверить что счетчик по индексу >= количеству вхождений остатков от хешей в массив по индексу

    def remove(self, item):
        hashes = get_hashes(item, self.size, self.hash_count)
        for hash in hashes:
            if self.bit_array[int(hash,2)] > 0:  # Защита от ухода в минус
                self.bit_array[int(hash,2)] -= 1
        self.element_count -= 1
    
    def union(self, other_filter):
        if self.size != other_filter.size or self.hash_count != other_filter.hash_count:
            raise ValueError("Фильтры должны иметь одинаковые размеры и количество хеш-функций.")

        # Создаем новый фильтр
        result_filter = BloomFilterCount(self.size, self.hash_count)
        result_filter.bit_array = [a | b for a, b in zip(self.bit_array, other_filter.bit_array)]
        result_filter.element_count = self.element_count + other_filter.element_count
        return result_filter

    def intersection(self, other_filter):
        if self.size != other_filter.size or self.hash_count != other_filter.hash_count:
            raise ValueError("Фильтры должны иметь одинаковые размеры и количество хеш-функций.")
        
        # Создаем новый фильтр
        result_filter = BloomFilterCount(self.size, self.hash_count)
        result_filter.bit_array = [a & b for a, b in zip(self.bit_array, other_filter.bit_array)]
        result_filter.element_count = min(self.element_count, other_filter.element_count)
        return result_filter
    
    def false_positive_probability(self):
        if self.size == 0 or self.element_count == 0:
            return 0
        
        # Формула для вероятности ложноположительных срабатываний
        m = self.size  # Размер битового массива
        n = self.element_count  # Количество добавленных элементов
        k = self.hash_count  # Количество хеш-функций
        
        argexp = -(k * n) / m
        probability = (1 - math.exp(argexp)) ** k
        return probability

In [4]:
counting_bloom_filter = BloomFilterCount(size = 64, hash_count=3)
# Добавляем несколько элементов
elements_to_add = ["apple", "banana", "grape"]
for element in elements_to_add:
    counting_bloom_filter.add(element)

# Проверяем элементы
test_elements = ["apple", "banana", 'grape', "blueberry"]
print("До удаления:")
for element in test_elements:
    result = counting_bloom_filter.check(element)
    print(f"Элемент '{element}' {'в множестве (возможно)' if result else 'не в множестве'}")

До удаления:
Элемент 'apple' в множестве (возможно)
Элемент 'banana' в множестве (возможно)
Элемент 'grape' в множестве (возможно)
Элемент 'blueberry' не в множестве


In [5]:
fp_prob_counting = counting_bloom_filter.false_positive_probability()
print(f"Вероятность ложноположительного срабатывания: {fp_prob_counting * 100:.2f}%")

Вероятность ложноположительного срабатывания: 0.23%


In [6]:
# Удаляем элемент
counting_bloom_filter.remove("apple")
# Проверяем снова
print("После удаления:")
for element in test_elements:
    result = counting_bloom_filter.check(element)
    print(f"Элемент '{element}' {'в множестве (возможно)' if result else 'не в множестве'}")

После удаления:
Элемент 'apple' не в множестве
Элемент 'banana' в множестве (возможно)
Элемент 'grape' в множестве (возможно)
Элемент 'blueberry' не в множестве


In [7]:
fp_prob_counting = counting_bloom_filter.false_positive_probability()
print(f"Вероятность ложноположительного срабатывания: {fp_prob_counting * 100:.2f}%")

Вероятность ложноположительного срабатывания: 0.07%
